## A Simple Text Classifier

#### Data loading and preprocessing


In [19]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf


In [20]:
data = pd.read_csv('data/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None) #default encoding is utf-8
data = data.sample(frac=1)
data.head(3)
# print(data[5][0])

,0,1,2,3,4,5
1531194,4,2177966647,Mon Jun 15 06:59:36 PDT 2009,NO_QUERY,PRINCESS_ACLF,IS SO GOIN GROCERY SHOPPING LATER. I LOVE FOOD...
756007,0,2288525608,Mon Jun 22 19:16:30 PDT 2009,NO_QUERY,mhernan31,@Nasamoah i was like omaaan.. and with the 8 k...
525846,0,2194060468,Tue Jun 16 09:17:27 PDT 2009,NO_QUERY,itsliambitch,uhh im sitting in ummm mcsalad waiting for my ...


In [21]:
# train-test split
train_size = int(0.7*len(data))
features = data[5]
targets = data[0]
X_train, X_test = features.values[:train_size], features.values[train_size:]
y_train, y_test = targets.values[:train_size], targets.values[train_size:]

y_train[y_train==2] = 1
y_train[y_train==4] = 2

y_test[y_test==2] = 1
y_test[y_test==4] = 2

In [23]:
# count vectorize them
count_vectorizer = CountVectorizer(max_features=100)
X_train_num = count_vectorizer.fit_transform(X_train).toarray()
X_test_num = count_vectorizer.transform(X_test).toarray()

In [24]:
# Inspect an instance
instance = 46544
print(X_train[instance])
print(X_train_num[instance])

@glynis_crawford Thank you for the #FF love! 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]


#### Text classification

In [25]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu), # x if x > 0 else alpha*x 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3)
])

adam_optimizer = tf.keras.optimizers.Adam()
model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
      'accuracy',
    ]
)

print(y_train)
model.fit(X_train_num, 
          y_train,
          batch_size=64,
          epochs=2,
          validation_split=0.1,
          verbose=1)

[2 0 0 ... 0 2 2]
Epoch 1/2
15750/15750 ━━━━━━━━━━━━━━━━━━━━ 56s 3ms/step - accuracy: 0.6527 - loss: 0.6255 - val_accuracy: 0.6683 - val_loss: 0.6058
Epoch 2/2
15750/15750 ━━━━━━━━━━━━━━━━━━━━ 49s 3ms/step - accuracy: 0.6630 - loss: 0.6098 - val_accuracy: 0.6687 - val_loss: 0.6035
